In [46]:
import warnings
warnings.filterwarnings('ignore')

In [47]:
from src.similarity_model import run_similarity_prediction
import pandas as pd
from tqdm import tqdm

# 1. Veri yükle
df = pd.read_csv("data/processed/match_odds_cleaned_20250801.csv")
df = df.sample(frac=1)


# 2. Sadece oran ve skor verisi eksiksiz olanları seç
filtered_df = df[
    df[[
        "totalHomeGoal", "totalAwayGoal",
        "Maç Sonucu :: MS 1", "Maç Sonucu :: MS 2", "Maç Sonucu :: MS X"
    ]].notna().all(axis=1)
].copy()

# 3. Değerlendirme fonksiyonu
def evaluate_prediction(result, true_home, true_away):
    """
    Tahmin edilen en yüksek EV'li outcome ile gerçek sonucu karşılaştırır ve oranı ekler.
    """
    if result is None:
        return None

    true_result = "MS1" if true_home > true_away else "MS2" if true_home < true_away else "MS X"
    best_outcome = max(result.items(), key=lambda x: x[1]["ev"] if x[1]["ev"] is not None else -999)[0]

    was_value = result[best_outcome]["is_value"]
    prediction_correct = best_outcome == true_result
    selected_odds = result[best_outcome]["odds"]

    return {
        "predicted_outcome": best_outcome,
        "true_outcome": true_result,
        "was_value": was_value,
        "correct": prediction_correct if was_value else None,
        "odds": selected_odds if was_value else None
    }

# 4. Value bet analiz döngüsü
ms_cols = [
    "Maç Sonucu :: MS 1",
    "Maç Sonucu :: MS 2",
    "Maç Sonucu :: MS X"
]

results = []
value_bet_limit = 100
value_bets_found = 0

print("🔄 100 value bet örneği bulunuyor...")

for _, row in tqdm(filtered_df.iterrows(), total=len(filtered_df)):
    match_id = row["match_id"]
    result = run_similarity_prediction(df, match_id=match_id, feature_cols=ms_cols, k=100)
    # print(result)
    evaluation = evaluate_prediction(result, row["totalHomeGoal"], row["totalAwayGoal"])
    # print(row)
    if evaluation and evaluation["was_value"]:
        results.append(evaluation)
        value_bets_found += 1

    if value_bets_found >= value_bet_limit:
        break

# 5. Sonuç analizi
df_results = pd.DataFrame(results)
correct_count = df_results["correct"].sum()
accuracy = round(correct_count / len(df_results), 3)

print(f"\n🎯 Value Bet Başarı Oranı: {accuracy * 100:.1f}% ({correct_count}/{len(df_results)})")
df_results.head()


🔄 100 value bet örneği bulunuyor...


 13%|█▎        | 380/2932 [00:05<00:39, 65.38it/s]


🎯 Value Bet Başarı Oranı: 41.0% (41/100)


,predicted_outcome,true_outcome,was_value,correct,odds
0,MS X,MS X,True,True,3.26
1,MS X,MS2,True,False,3.53
2,MS2,MS2,True,True,2.04
3,MS2,MS2,True,True,6.09
4,MS X,MS X,True,True,3.16


In [48]:
df_results = pd.DataFrame(results)
correct_count = df_results["correct"].sum()
accuracy = round(correct_count / len(df_results), 3)

# Sadece başarılı value bet'lerin oranlarını topla
total_return = df_results[df_results["correct"] == True]["odds"].sum()

print(f"\n🎯 Value Bet Başarı Oranı: {accuracy * 100:.1f}% ({correct_count}/{len(df_results)})")
print(f"💰 Doğru value bet'lerin toplam getirisi: {round(total_return, 2)} birim")

# İlk birkaç sonucu göster
df_results.head()



🎯 Value Bet Başarı Oranı: 41.0% (41/100)
💰 Doğru value bet'lerin toplam getirisi: 121.11 birim


,predicted_outcome,true_outcome,was_value,correct,odds
0,MS X,MS X,True,True,3.26
1,MS X,MS2,True,False,3.53
2,MS2,MS2,True,True,2.04
3,MS2,MS2,True,True,6.09
4,MS X,MS X,True,True,3.16
